In [1]:
#!pip3 install pysam

In [2]:
##sudo apt install samtools

In [3]:
##for f in *.bam ; do samtools index -@4 $f ; done

In [4]:
import pysam

In [5]:
import os

In [6]:
def countMappingGene(gff, bams=None, area=.9, readSize=0, folder=True, bam=None, out='gene_counts.tsv'):
    
    print('parsing gff...')
    genes = [x.split('\t') for x in open(gff).readlines() if '\tgene\t' in x]
    
    print('parsing %d genes...' % len(genes))
    gdata = {
        g[8].split('ID=')[1].split(';')[0]:
        [g[0], int(g[3]), int(g[4])]
        for g in genes
    }
    
    for gene, value in gdata.items():
        glen = value[2]-value[1]+1
        part = readSize if readSize > 0 else (((1-area) / 2) * glen)
        if glen <= part*2:
            part = ((1-area) / 2) * glen
        value[1] = int(value[1] + part)
        value[2] = int(value[2] - part)
    
    result = {} 
    if not bam and not bams:
        raise Exception("bam or bams must be taked")
    bams = [bam] if bam else ([bams+"/"+b for b in os.listdir(bams) if b.endswith('bam')] if folder else bams)
    for bam in bams:
        print('processing', bam)
        samfile = pysam.AlignmentFile(bam, "rb")
        result[bam] = {
            gene: len(list(samfile.fetch(v[0], v[1], v[2])))
            for gene, v in gdata.items()
        }
    if readSize > 0:
        print('Results for mapped on gene len - readSize(%d)' % readSize)
    else:
        print('Results for mapped on least %d%% of gene len' % (100*area))
    print(
        {f: "%.2f%%" % (100*len([x for x in g.values() if x])/len(g)) for f, g in result.items()}
    )
    ls = [['gene'] + list(result)] + [[g] + [str(result[f][g]) for f in result] for g in gdata]
    open(out, 'w').writelines(['\t'.join(l) + '\n' for l in ls])
    return result, ls
    

In [7]:
res = countMappingGene('../Ca_scaffolds.gff3', 'bams/', readSize=101)

parsing gff...
parsing 78311 genes...
processing bams//ER35_Aligned.sortedByCoord.out.bam
processing bams//ER36_Aligned.sortedByCoord.out.bam
processing bams//ER37_Aligned.sortedByCoord.out.bam
processing bams//ER38_Aligned.sortedByCoord.out.bam
processing bams//ER39_Aligned.sortedByCoord.out.bam
processing bams//ER40_Aligned.sortedByCoord.out.bam
processing bams//ER41_Aligned.sortedByCoord.out.bam
processing bams//ER42_Aligned.sortedByCoord.out.bam
processing bams//ER43_Aligned.sortedByCoord.out.bam
Results for mapped on gene len - readSize(101)
{'bams//ER35_Aligned.sortedByCoord.out.bam': '70.93%', 'bams//ER36_Aligned.sortedByCoord.out.bam': '66.24%', 'bams//ER37_Aligned.sortedByCoord.out.bam': '71.17%', 'bams//ER38_Aligned.sortedByCoord.out.bam': '71.14%', 'bams//ER39_Aligned.sortedByCoord.out.bam': '77.94%', 'bams//ER40_Aligned.sortedByCoord.out.bam': '74.50%', 'bams//ER41_Aligned.sortedByCoord.out.bam': '71.29%', 'bams//ER42_Aligned.sortedByCoord.out.bam': '67.84%', 'bams//ER43_Al